In [1]:
from utils.utils import loadData,encode,transform,getSeq,getUserAvgSeq,getUserMaxSeq,upload

from model.W2V import wvmodel

from model.LGB import base_predict,base_train

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split


method = "combine_seq"

In [2]:
# 加载数据
df_ad,df_click_log,df_user,df_click_log_test = loadData("./train/",mode="pro")

Step1:Loading Data......


In [4]:
cid = df_click_log_test["creative_id"].tolist() + df_click_log["creative_id"].tolist()

In [ ]:
def process (base="",key = "",isSort=True,sortKey="",isCombine=False):
    cid = df_click_log_test[key].tolist() + df_click_log[key].tolist()
    table,_table = encode([cid],isProNaN = True)
    df_click_log[key] = transform(df_click_log[key],table)
    df_click_log_test[key] = transform(df_click_log_test[key],table)
    dtrain = getSeq (df_click_log,key=base,val=key,isSort=isSort,sortKey=sortKey)
    dtest = getSeq(df_click_log_test,key=base,val=key,isSort=isSort,sortKey=sortKey)
    
    if isCombine:
        cid_seq = dtrain_total[key+"_seq"].tolist() + dtest_total[key+"_seq"].tolist()
        path = "./" + key + "_combine.w2v"
    else :
        cid_seq = dtrain_total[key+"_seq"].tolist()
        path = "./" + key + ".w2v"

    params = {"min_count":1,
          "window" : 5,
          "size":128,
          "workers":16}

    weight = wvmodel(cid_seq,10,"./test_nn_seq/" + key + ".w2v",params)
    
    return weight,dtrain,dtest

In [5]:
table,_table = encode([cid],isProNaN = True)

In [6]:
df_click_log["creative_id"] = transform(df_click_log["creative_id"],table)

In [7]:
# T1 ：测试集中cid也用作编码

df_click_log_test["creative_id"] = transform(df_click_log_test["creative_id"],table)

In [9]:
# 异常耗时  后续可以用spark实现

dtrain = getSeq (df_click_log,key="user_id",val="creative_id",isSort=True,sortKey="time")

In [10]:
# 异常耗时  后续可以用spark实现

dtest = getSeq(df_click_log_test,key="user_id",val="creative_id",isSort=True,sortKey="time")

In [12]:
dtrain_total = dtrain.merge(df_user, how='left', on="user_id")
dtest_total = dtest.merge(df_user, how='left', on="user_id")

In [14]:
dtrain_total['gender'] = dtrain_total['gender'] - 1
dtrain_total['age'] = dtrain_total['age'] - 1

In [15]:
#import pickle
#pickle.dump(dtrain_total,open(method + "_data_cid.train"))
#pickle.dump(dtest_total,open(method + "_data_cid.test"))
#dtrain_total=pickle.load(open(method + "_data_cid.train"))
#dtest_total=pickle.load(open(method + "_data_cid.test"))
dtrain_total.to_csv(method + "_data_cid.train")
dtest_total.to_csv(method + "_data_cid.test")

In [16]:
#dtrain_total = pd.read_csv(method + "_data_cid.train")
#dtest_total = pd.read_csv(method + "_data_cid.test")
#dtrain_total["creative_id_seq"] = dtrain_total["creative_id_seq"].map(lambda x : list(eval(x)))
#dtest_total["creative_id_seq"] = dtest_total["creative_id_seq"].map(lambda x : list(eval(x)))

In [ ]:
def process (base="",key = "",isSort=True,sortKey="",isCombine=False):
    
    
    table,_table = encode([df_ad[key].tolist()],isProNaN = True)
    df_click_log[key] = transform(df_click_log[key],table)
    df_click_log_test[key] = transform(df_click_log_test[key],table)
    dtrain = getSeq (df_click_log,key=base,val=key,isSort=isSort,sortKey=sortKey)
    dtest = getSeq(df_click_log_test,key=base,val=key,isSort=isSort,sortKey=sortKey)
    
    if isCombine:
        cid_seq = dtrain_total[key+"_seq"].tolist() + dtest_total[key+"_seq"].tolist()
        path = "./" + key + "_combine.w2v"
    else :
        cid_seq = dtrain_total[key+"_seq"].tolist()
        path = "./" + key + ".w2v"

    params = {"min_count":1,
          "window" : 5,
          "size":128,
          "workers":16}

    weight = wvmodel(cid_seq,10,"./test_nn_seq/" + key + ".w2v",params)
    
    return weight,dtrain,dtest

In [17]:
# T2 ：测试集序列也去学习embedding

cid_seq = dtrain_total["creative_id_seq"].tolist() + dtest_total["creative_id_seq"].tolist()

params = {"min_count":1,  # 保留的最小出现频次
          "alpha":0.1,       # 初始学习率
          "seed":2020,
          "min_alpha":0.01,   # 随着学习进行，学习率线性下降到这个最小数
          "window" : 4,   # 当前和预测单词之间的最大距离
          "size":128,     # 生成词向量的维度
          "compute_loss":False,# 损失(loss)值，如果是True 就会保存
          "workers":8}   # 几个CPU进行跑

model = wvmodel(cid_seq,20,outpath='./combineTest128/combineTest128.wv',params = params)

#import gensim

#model = gensim.models.word2vec.Word2Vec.load("creative.w2v")

weight = model.wv

**********Word2Vector*********
**********initing*********
**********training*********
**********saving*********


In [18]:
seq = dtrain_total["creative_id_seq"].tolist()
data_avg = getUserAvgSeq(seq,weight)
data_max = getUserMaxSeq(seq,weight)

In [19]:
data = [np.append(x,y) for x,y in zip(data_avg,data_max)]

In [20]:
print("finished!")
ddf = pd.DataFrame(np.array(data))
ddf["age"] = dtrain_total['age']
ddf["gender"] = dtrain_total['gender']
print("finished!")

# 切分数据
df_train, df_val = train_test_split(ddf,test_size=0.2, random_state=2020)
print("finished!")
y_train_gender = df_train['gender']
y_train_age = df_train['age']
x_train = df_train.drop(['gender', 'age'], axis=1)
y_val_gender = df_val['gender']
y_val_age = df_val['age']
x_val = df_val.drop(['gender', 'age'], axis=1)
print("The shape of train is : ",len(x_train))
print("The shape of val is : ",len(x_val))

finished!
finished!
finished!
The shape of train is :  720000
The shape of val is :  180000


In [21]:
%%time
params = {
        'max_depth':5,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves':31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 50,
        "use_missing": False,
        "boost_from_average": False,
        "n_jobs": 10,
        "device":"cpu",
        "save_binary":True
        }
gbm_gender = base_train(x_train, y_train_gender, x_val, y_val_gender,params,job='classification')

Starting training...
[1]	valid_0's binary_logloss: 0.670327
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.650697
[3]	valid_0's binary_logloss: 0.632366
[4]	valid_0's binary_logloss: 0.614568
[5]	valid_0's binary_logloss: 0.598545
[6]	valid_0's binary_logloss: 0.583635
[7]	valid_0's binary_logloss: 0.569412
[8]	valid_0's binary_logloss: 0.556189
[9]	valid_0's binary_logloss: 0.544256
[10]	valid_0's binary_logloss: 0.532817
[11]	valid_0's binary_logloss: 0.522175
[12]	valid_0's binary_logloss: 0.512324
[13]	valid_0's binary_logloss: 0.50264
[14]	valid_0's binary_logloss: 0.493708
[15]	valid_0's binary_logloss: 0.485333
[16]	valid_0's binary_logloss: 0.477328
[17]	valid_0's binary_logloss: 0.469706
[18]	valid_0's binary_logloss: 0.4625
[19]	valid_0's binary_logloss: 0.45568
[20]	valid_0's binary_logloss: 0.449225
[21]	valid_0's binary_logloss: 0.443084
[22]	valid_0's binary_logloss: 0.437136
[23]	valid_0's binary_logloss: 0.431542
[24]	valid

[203]	valid_0's binary_logloss: 0.255155
[204]	valid_0's binary_logloss: 0.255002
[205]	valid_0's binary_logloss: 0.254853
[206]	valid_0's binary_logloss: 0.254705
[207]	valid_0's binary_logloss: 0.254568
[208]	valid_0's binary_logloss: 0.254421
[209]	valid_0's binary_logloss: 0.254278
[210]	valid_0's binary_logloss: 0.254142
[211]	valid_0's binary_logloss: 0.25399
[212]	valid_0's binary_logloss: 0.253873
[213]	valid_0's binary_logloss: 0.253744
[214]	valid_0's binary_logloss: 0.253615
[215]	valid_0's binary_logloss: 0.253496
[216]	valid_0's binary_logloss: 0.25337
[217]	valid_0's binary_logloss: 0.253247
[218]	valid_0's binary_logloss: 0.25312
[219]	valid_0's binary_logloss: 0.252998
[220]	valid_0's binary_logloss: 0.252896
[221]	valid_0's binary_logloss: 0.252773
[222]	valid_0's binary_logloss: 0.25265
[223]	valid_0's binary_logloss: 0.252521
[224]	valid_0's binary_logloss: 0.252382
[225]	valid_0's binary_logloss: 0.252276
[226]	valid_0's binary_logloss: 0.252175
[227]	valid_0's bina

[404]	valid_0's binary_logloss: 0.240357
[405]	valid_0's binary_logloss: 0.240311
[406]	valid_0's binary_logloss: 0.240268
[407]	valid_0's binary_logloss: 0.240228
[408]	valid_0's binary_logloss: 0.240202
[409]	valid_0's binary_logloss: 0.240161
[410]	valid_0's binary_logloss: 0.240125
[411]	valid_0's binary_logloss: 0.240084
[412]	valid_0's binary_logloss: 0.240051
[413]	valid_0's binary_logloss: 0.24001
[414]	valid_0's binary_logloss: 0.239956
[415]	valid_0's binary_logloss: 0.239922
[416]	valid_0's binary_logloss: 0.239896
[417]	valid_0's binary_logloss: 0.239848
[418]	valid_0's binary_logloss: 0.239805
[419]	valid_0's binary_logloss: 0.239772
[420]	valid_0's binary_logloss: 0.239732
[421]	valid_0's binary_logloss: 0.239695
[422]	valid_0's binary_logloss: 0.239665
[423]	valid_0's binary_logloss: 0.23963
[424]	valid_0's binary_logloss: 0.239602
[425]	valid_0's binary_logloss: 0.239563
[426]	valid_0's binary_logloss: 0.239531
[427]	valid_0's binary_logloss: 0.239487
[428]	valid_0's bi

[607]	valid_0's binary_logloss: 0.234719
[608]	valid_0's binary_logloss: 0.23469
[609]	valid_0's binary_logloss: 0.234685
[610]	valid_0's binary_logloss: 0.23467
[611]	valid_0's binary_logloss: 0.23465
[612]	valid_0's binary_logloss: 0.234625
[613]	valid_0's binary_logloss: 0.234594
[614]	valid_0's binary_logloss: 0.23457
[615]	valid_0's binary_logloss: 0.234558
[616]	valid_0's binary_logloss: 0.23454
[617]	valid_0's binary_logloss: 0.234515
[618]	valid_0's binary_logloss: 0.234496
[619]	valid_0's binary_logloss: 0.234485
[620]	valid_0's binary_logloss: 0.234463
[621]	valid_0's binary_logloss: 0.234443
[622]	valid_0's binary_logloss: 0.234416
[623]	valid_0's binary_logloss: 0.234402
[624]	valid_0's binary_logloss: 0.234386
[625]	valid_0's binary_logloss: 0.234366
[626]	valid_0's binary_logloss: 0.234345
[627]	valid_0's binary_logloss: 0.234316
[628]	valid_0's binary_logloss: 0.234302
[629]	valid_0's binary_logloss: 0.234284
[630]	valid_0's binary_logloss: 0.234266
[631]	valid_0's binar

[810]	valid_0's binary_logloss: 0.231551
[811]	valid_0's binary_logloss: 0.231539
[812]	valid_0's binary_logloss: 0.231532
[813]	valid_0's binary_logloss: 0.231518
[814]	valid_0's binary_logloss: 0.231516
[815]	valid_0's binary_logloss: 0.231512
[816]	valid_0's binary_logloss: 0.231506
[817]	valid_0's binary_logloss: 0.231491
[818]	valid_0's binary_logloss: 0.23148
[819]	valid_0's binary_logloss: 0.23147
[820]	valid_0's binary_logloss: 0.231461
[821]	valid_0's binary_logloss: 0.231439
[822]	valid_0's binary_logloss: 0.231432
[823]	valid_0's binary_logloss: 0.231421
[824]	valid_0's binary_logloss: 0.231395
[825]	valid_0's binary_logloss: 0.231387
[826]	valid_0's binary_logloss: 0.231372
[827]	valid_0's binary_logloss: 0.231357
[828]	valid_0's binary_logloss: 0.231348
[829]	valid_0's binary_logloss: 0.231328
[830]	valid_0's binary_logloss: 0.231317
[831]	valid_0's binary_logloss: 0.231308
[832]	valid_0's binary_logloss: 0.231292
[833]	valid_0's binary_logloss: 0.231284
[834]	valid_0's bi

...

[1601]	valid_0's binary_logloss: 0.226109
[1602]	valid_0's binary_logloss: 0.226104
[1603]	valid_0's binary_logloss: 0.2261
[1604]	valid_0's binary_logloss: 0.226099
[1605]	valid_0's binary_logloss: 0.226105
[1606]	valid_0's binary_logloss: 0.226094
[1607]	valid_0's binary_logloss: 0.226081
[1608]	valid_0's binary_logloss: 0.226083
[1609]	valid_0's binary_logloss: 0.226078
[1610]	valid_0's binary_logloss: 0.226074
[1611]	valid_0's binary_logloss: 0.226068
[1612]	valid_0's binary_logloss: 0.226065
[1613]	valid_0's binary_logloss: 0.226064
[1614]	valid_0's binary_logloss: 0.226063
[1615]	valid_0's binary_logloss: 0.226058
[1616]	valid_0's binary_logloss: 0.226058
[1617]	valid_0's binary_logloss: 0.226054
[1618]	valid_0's binary_logloss: 0.226055
[1619]	valid_0's binary_logloss: 0.226046
[1620]	valid_0's binary_logloss: 0.226044
[1621]	valid_0's binary_logloss: 0.226042
[1622]	valid_0's binary_logloss: 0.226039
[1623]	valid_0's binary_logloss: 0.226035
[1624]	valid_0's binary_logloss: 0.2

[1799]	valid_0's binary_logloss: 0.225413
[1800]	valid_0's binary_logloss: 0.225411
[1801]	valid_0's binary_logloss: 0.22541
[1802]	valid_0's binary_logloss: 0.225409
[1803]	valid_0's binary_logloss: 0.225405
[1804]	valid_0's binary_logloss: 0.225404
[1805]	valid_0's binary_logloss: 0.225396
[1806]	valid_0's binary_logloss: 0.225392
[1807]	valid_0's binary_logloss: 0.22539
[1808]	valid_0's binary_logloss: 0.225392
[1809]	valid_0's binary_logloss: 0.225382
[1810]	valid_0's binary_logloss: 0.225379
[1811]	valid_0's binary_logloss: 0.225365
[1812]	valid_0's binary_logloss: 0.225364
[1813]	valid_0's binary_logloss: 0.225366
[1814]	valid_0's binary_logloss: 0.225359
[1815]	valid_0's binary_logloss: 0.225362
[1816]	valid_0's binary_logloss: 0.225359
[1817]	valid_0's binary_logloss: 0.225354
[1818]	valid_0's binary_logloss: 0.225349
[1819]	valid_0's binary_logloss: 0.225349
[1820]	valid_0's binary_logloss: 0.225342
[1821]	valid_0's binary_logloss: 0.225342
[1822]	valid_0's binary_logloss: 0.2

[1996]	valid_0's binary_logloss: 0.224809
[1997]	valid_0's binary_logloss: 0.22481
[1998]	valid_0's binary_logloss: 0.224806
[1999]	valid_0's binary_logloss: 0.224801
[2000]	valid_0's binary_logloss: 0.224802
[2001]	valid_0's binary_logloss: 0.2248
[2002]	valid_0's binary_logloss: 0.224801
[2003]	valid_0's binary_logloss: 0.224801
[2004]	valid_0's binary_logloss: 0.224801
[2005]	valid_0's binary_logloss: 0.224797
[2006]	valid_0's binary_logloss: 0.224793
[2007]	valid_0's binary_logloss: 0.224788
[2008]	valid_0's binary_logloss: 0.224786
[2009]	valid_0's binary_logloss: 0.224782
[2010]	valid_0's binary_logloss: 0.224786
[2011]	valid_0's binary_logloss: 0.224782
[2012]	valid_0's binary_logloss: 0.224779
[2013]	valid_0's binary_logloss: 0.224778
[2014]	valid_0's binary_logloss: 0.224766
[2015]	valid_0's binary_logloss: 0.224766
[2016]	valid_0's binary_logloss: 0.22476
[2017]	valid_0's binary_logloss: 0.224756
[2018]	valid_0's binary_logloss: 0.224756
[2019]	valid_0's binary_logloss: 0.224

[2193]	valid_0's binary_logloss: 0.224319
[2194]	valid_0's binary_logloss: 0.224318
[2195]	valid_0's binary_logloss: 0.224314
[2196]	valid_0's binary_logloss: 0.224309
[2197]	valid_0's binary_logloss: 0.224308
[2198]	valid_0's binary_logloss: 0.224313
[2199]	valid_0's binary_logloss: 0.224308
[2200]	valid_0's binary_logloss: 0.22431
[2201]	valid_0's binary_logloss: 0.224307
[2202]	valid_0's binary_logloss: 0.224302
[2203]	valid_0's binary_logloss: 0.224299
[2204]	valid_0's binary_logloss: 0.224297
[2205]	valid_0's binary_logloss: 0.224296
[2206]	valid_0's binary_logloss: 0.224297
[2207]	valid_0's binary_logloss: 0.224295
[2208]	valid_0's binary_logloss: 0.224296
[2209]	valid_0's binary_logloss: 0.224294
[2210]	valid_0's binary_logloss: 0.224294
[2211]	valid_0's binary_logloss: 0.224292
[2212]	valid_0's binary_logloss: 0.224293
[2213]	valid_0's binary_logloss: 0.224292
[2214]	valid_0's binary_logloss: 0.224287
[2215]	valid_0's binary_logloss: 0.224288
[2216]	valid_0's binary_logloss: 0.

[2390]	valid_0's binary_logloss: 0.223774
[2391]	valid_0's binary_logloss: 0.223774
[2392]	valid_0's binary_logloss: 0.223771
[2393]	valid_0's binary_logloss: 0.223769
[2394]	valid_0's binary_logloss: 0.223767
[2395]	valid_0's binary_logloss: 0.223762
[2396]	valid_0's binary_logloss: 0.223762
[2397]	valid_0's binary_logloss: 0.223764
[2398]	valid_0's binary_logloss: 0.223761
[2399]	valid_0's binary_logloss: 0.223762
[2400]	valid_0's binary_logloss: 0.223767
[2401]	valid_0's binary_logloss: 0.22376
[2402]	valid_0's binary_logloss: 0.223752
[2403]	valid_0's binary_logloss: 0.223746
[2404]	valid_0's binary_logloss: 0.223735
[2405]	valid_0's binary_logloss: 0.223736
[2406]	valid_0's binary_logloss: 0.223728
[2407]	valid_0's binary_logloss: 0.223725
[2408]	valid_0's binary_logloss: 0.223724
[2409]	valid_0's binary_logloss: 0.22372
[2410]	valid_0's binary_logloss: 0.223719
[2411]	valid_0's binary_logloss: 0.223718
[2412]	valid_0's binary_logloss: 0.223717
[2413]	valid_0's binary_logloss: 0.2

In [22]:
%%time
params = {
    'max_depth':5,
    'boosting_type': 'gbdt',
    'objective': 'multiclass',  
    #'objective': 'regression',
    'num_class': 10,  
    'metric': 'multi_error', 
    #'metric':['l1','l2'],
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 2,
    "n_jobs": 10,
    "device":"cpu",
    "save_binary":True
}
gbm_age = base_train(x_train, y_train_age, x_val, y_val_age,params,job='multi')

Starting training...
[1]	valid_0's multi_error: 0.773239
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_error: 0.773239
[3]	valid_0's multi_error: 0.773239
[4]	valid_0's multi_error: 0.773239
[5]	valid_0's multi_error: 0.773239
[6]	valid_0's multi_error: 0.773222
[7]	valid_0's multi_error: 0.773133
[8]	valid_0's multi_error: 0.771678
[9]	valid_0's multi_error: 0.769956
[10]	valid_0's multi_error: 0.767539
[11]	valid_0's multi_error: 0.764961
[12]	valid_0's multi_error: 0.763011
[13]	valid_0's multi_error: 0.760833
[14]	valid_0's multi_error: 0.758739
[15]	valid_0's multi_error: 0.756667
[16]	valid_0's multi_error: 0.754567
[17]	valid_0's multi_error: 0.752622
[18]	valid_0's multi_error: 0.750683
[19]	valid_0's multi_error: 0.748644
[20]	valid_0's multi_error: 0.747044
[21]	valid_0's multi_error: 0.744794
[22]	valid_0's multi_error: 0.742517
[23]	valid_0's multi_error: 0.740467
[24]	valid_0's multi_error: 0.738611
[25]	valid_0's multi_error: 0.736706
[

[219]	valid_0's multi_error: 0.659556
[220]	valid_0's multi_error: 0.659317
[221]	valid_0's multi_error: 0.6593
[222]	valid_0's multi_error: 0.659267
[223]	valid_0's multi_error: 0.658883
[224]	valid_0's multi_error: 0.658972
[225]	valid_0's multi_error: 0.659056
[226]	valid_0's multi_error: 0.659011
[227]	valid_0's multi_error: 0.658828
[228]	valid_0's multi_error: 0.658717
[229]	valid_0's multi_error: 0.658706
[230]	valid_0's multi_error: 0.658472
[231]	valid_0's multi_error: 0.658394
[232]	valid_0's multi_error: 0.658306
[233]	valid_0's multi_error: 0.657894
[234]	valid_0's multi_error: 0.657594
[235]	valid_0's multi_error: 0.657406
[236]	valid_0's multi_error: 0.657506
[237]	valid_0's multi_error: 0.657444
[238]	valid_0's multi_error: 0.657389
[239]	valid_0's multi_error: 0.657222
[240]	valid_0's multi_error: 0.657083
[241]	valid_0's multi_error: 0.657
[242]	valid_0's multi_error: 0.656867
[243]	valid_0's multi_error: 0.656767
[244]	valid_0's multi_error: 0.656661
[245]	valid_0's m

[436]	valid_0's multi_error: 0.644867
[437]	valid_0's multi_error: 0.644983
[438]	valid_0's multi_error: 0.645033
[439]	valid_0's multi_error: 0.644967
[440]	valid_0's multi_error: 0.644817
[441]	valid_0's multi_error: 0.644778
[442]	valid_0's multi_error: 0.644706
[443]	valid_0's multi_error: 0.644689
[444]	valid_0's multi_error: 0.644644
[445]	valid_0's multi_error: 0.644644
[446]	valid_0's multi_error: 0.644617
[447]	valid_0's multi_error: 0.644583
[448]	valid_0's multi_error: 0.644517
[449]	valid_0's multi_error: 0.644539
[450]	valid_0's multi_error: 0.64455
[451]	valid_0's multi_error: 0.644428
[452]	valid_0's multi_error: 0.6445
[453]	valid_0's multi_error: 0.644383
[454]	valid_0's multi_error: 0.644239
[455]	valid_0's multi_error: 0.644122
[456]	valid_0's multi_error: 0.644078
[457]	valid_0's multi_error: 0.644233
[458]	valid_0's multi_error: 0.644206
[459]	valid_0's multi_error: 0.644172
[460]	valid_0's multi_error: 0.644094
[461]	valid_0's multi_error: 0.644139
[462]	valid_0's

In [33]:
result = base_predict(gbm_age,gbm_gender,dtest_total,weight,isSave=True,method="avg_max")

ValueError: Length of values does not match length of index

In [13]:
upload("submission.csv")

{'Content-Length': '0', 'Connection': 'keep-alive', 'Date': 'Tue, 12 May 2020 15:17:15 GMT', 'ETag': '"3f8929f8fa238178c7c5491f9e06326a"', 'Server': 'tencent-cos', 'x-cos-hash-crc64ecma': '17821355056839019512', 'x-cos-request-id': 'NWViYWJkZmFfYTQxZDFkMDlfYTRiNV9hMGU5Yg=='}


In [ ]:
# 人工统计特征 + lgb  loss = 0.6   acc = 0.841

# word2vector(0.2 cid=avg128) + lgb  loss=0.22 acc = 1.244

# word2vector(0.2 cid=avg128 + test) + lgb acc = 0.21 0.64 acc = 1.275

# word2vector(0.05 cid=avg256 + test) + lgb loss = 0.208 0.636 acc = 1.281

# word2vector(0.05 cid=avg256+max256 + test) + lgb loss = 